In [3]:
:dep chrono = { version = "0.4.31"}

In [23]:
:dep polars = { version = "0.35.0", features = ["describe", "lazy", "ndarray", "object", "dtype-struct", "concat_str", "mode", "rows"] }

In [5]:
use polars::datatypes::DataType;
use polars::prelude::*;
use chrono::{DateTime, NaiveDateTime, TimeZone, Utc};
use std::sync::Arc;

In [6]:
let series1: Series = [1, 2, 3].iter().collect();
// or
// let series1: Series = Series::new("", &[1, 2, 3]);
series1

shape: (3,)
Series: '' [i32]
[
	1
	2
	3
]

In [7]:
use std::fs::File;

use chrono::prelude::*;
use polars::prelude::*;

let mut df: DataFrame = df!(
    "integer" => &[1, 2, 3],
    "date" => &[
            NaiveDate::from_ymd_opt(2025, 1, 1).unwrap().and_hms_opt(0, 0, 0).unwrap(),
            NaiveDate::from_ymd_opt(2025, 1, 2).unwrap().and_hms_opt(0, 0, 0).unwrap(),
            NaiveDate::from_ymd_opt(2025, 1, 3).unwrap().and_hms_opt(0, 0, 0).unwrap(),
    ],
    "float" => &[4.0, 5.0, 6.0],
    "string" => &["a", "b", "c"],
)
.unwrap();
println!("{}", df);

shape: (3, 4)
┌─────────┬─────────────────────┬───────┬────────┐
│ integer ┆ date                ┆ float ┆ string │
│ ---     ┆ ---                 ┆ ---   ┆ ---    │
│ i32     ┆ datetime[ms]        ┆ f64   ┆ str    │
╞═════════╪═════════════════════╪═══════╪════════╡
│ 1       ┆ 2025-01-01 00:00:00 ┆ 4.0   ┆ a      │
│ 2       ┆ 2025-01-02 00:00:00 ┆ 5.0   ┆ b      │
│ 3       ┆ 2025-01-03 00:00:00 ┆ 6.0   ┆ c      │
└─────────┴─────────────────────┴───────┴────────┘


In [8]:
let seasons_ser: Series = Series::new("seasons", &["Winter", "Spring", "Summer", "Fall"]);
seasons_ser

shape: (4,)
Series: 'seasons' [str]
[
	"Winter"
	"Spring"
	"Summer"
	"Fall"
]

In [10]:
let s: Series = Series::new("numbers", &[Some(f64::NAN), Some(1.), Some(2.)]);
s.cast(&DataType::Int64).unwrap()

shape: (3,)
Series: 'numbers' [i64]
[
	null
	1
	2
]

In [11]:
let date: DateTime<Utc> = Utc.with_ymd_and_hms(2020, 1, 1, 0, 0, 0).unwrap();
let s = Series::new("b", &[date.date_naive()]);
s

shape: (1,)
Series: 'b' [date]
[
	2020-01-01
]

In [12]:
date

2020-01-01T00:00:00Z

In [14]:
let x = Utc.with_ymd_and_hms(2020, 1, 1, 0, 0, 0);

In [15]:
x

Single(2020-01-01T00:00:00Z)

In [16]:
let s: Series = [1, 2, 3].iter().collect();
let s_squared: Series = s.i32()
     .expect("datatype mismatch")
     .into_iter()
     .map(|optional_v| {
         match optional_v {
             Some(v) => Some(v * v),
             None => None, // null value
         }
 }).collect();

In [18]:
s_squared

shape: (3,)
Series: '' [i32]
[
	1
	4
	9
]

In [19]:
use polars::prelude::*;


    // Create a DataFrame
    let df: DataFrame = DataFrame::new(vec![
        Series::new("A", vec![1, 2, 3]),
        Series::new("B", vec![4, 5, 6]),
    ]).unwrap();

    // Iterate over the DataFrame rows
    for row in df.iter() {
        println!("{:?}", row);
    }


shape: (3,)
Series: 'A' [i32]
[
	1
	2
	3
]
shape: (3,)
Series: 'B' [i32]
[
	4
	5
	6
]


()

In [26]:
use polars::prelude::*;

fn main() {

    let col1 = Series::new("col1", vec![1, 2, 3, 4, 5]);
    let col2 = Series::new("col2", vec![5, 4, 3, 2, 1]);
    let col3 = Series::new("col3", vec![1, 2, 3, 4, 5]);
    let mut df = DataFrame::new(vec![col1, col2, col3]).unwrap();

    let mut signal = vec![0; df.height()];
    
    // Iterate over DataFrame and update signal
    for i in 0..df.height() {
        if df.column("col1").unwrap().get(i).unwrap() == AnyValue::Int32(3) 
            && df.column("col2").unwrap().get(i).unwrap() == AnyValue::Int32(3)
            && df.column("col3").unwrap().get(i).unwrap() == AnyValue::Int32(3) 
        {
            signal[i] = 1;
        }
    }
    
    // Add new column to DataFrame
    let new_col = Series::new("signal", signal.clone());
    
    let df = df.with_column(new_col).unwrap();
    println!("after: {:?}", df);
}

In [27]:
main()

after: shape: (5, 4)
┌──────┬──────┬──────┬────────┐
│ col1 ┆ col2 ┆ col3 ┆ signal │
│ ---  ┆ ---  ┆ ---  ┆ ---    │
│ i32  ┆ i32  ┆ i32  ┆ i32    │
╞══════╪══════╪══════╪════════╡
│ 1    ┆ 5    ┆ 1    ┆ 0      │
│ 2    ┆ 4    ┆ 2    ┆ 0      │
│ 3    ┆ 3    ┆ 3    ┆ 1      │
│ 4    ┆ 2    ┆ 4    ┆ 0      │
│ 5    ┆ 1    ┆ 5    ┆ 0      │
└──────┴──────┴──────┴────────┘


()

In [28]:
use polars::prelude::*;

fn main2() {

    let col1 = Series::new("col1", vec![1, 2, 3, 4, 5]);
    let col2 = Series::new("col2", vec![5, 4, 3, 2, 1]);
    let col3 = Series::new("col3", vec![1, 2, 3, 4, 5]);
    let mut df = DataFrame::new(vec![col1, col2, col3]).unwrap();

    let expr = (col("col1").eq(3))
        .and(col("col2").eq(3))
        .and(col("col3").eq(3));
    println!(
        "{:?}",
        df.clone()
            .lazy()
            .select([
                col("*"),
                expr.clone().alias("col1 == col2 == col3 == 3"),
                when(expr).then(1).otherwise(0).alias("signal")
            ])
            .collect()
            .unwrap()
    );
}

In [29]:
main2()

shape: (5, 5)
┌──────┬──────┬──────┬───────────────────────────┬────────┐
│ col1 ┆ col2 ┆ col3 ┆ col1 == col2 == col3 == 3 ┆ signal │
│ ---  ┆ ---  ┆ ---  ┆ ---                       ┆ ---    │
│ i32  ┆ i32  ┆ i32  ┆ bool                      ┆ i32    │
╞══════╪══════╪══════╪═══════════════════════════╪════════╡
│ 1    ┆ 5    ┆ 1    ┆ false                     ┆ 0      │
│ 2    ┆ 4    ┆ 2    ┆ false                     ┆ 0      │
│ 3    ┆ 3    ┆ 3    ┆ true                      ┆ 1      │
│ 4    ┆ 2    ┆ 4    ┆ false                     ┆ 0      │
│ 5    ┆ 1    ┆ 5    ┆ false                     ┆ 0      │
└──────┴──────┴──────┴───────────────────────────┴────────┘


()